Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

In [2]:
#Update your token
STUDENT_TOKEN = 'KATHERINE DUQUE'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


In [3]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [4]:
#this function returns the empty cells found in a board. 
def returnEmptyPosition(board):
    emptyCells = []
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] in ['.']:
                emptyCells.append((i,j))
                
    return emptyCells

In [5]:
#iterative version of program 
#itertools is a python library that automates sum processes like what we 
#need to calculate the MRV. 
import itertools

#isNotDigit determines if a place in the board is a digit or not.
#since a value in aboard can only be a wall, an empty space, or an actual 
#digit this function helps determine where a candidate can be placed
def isNotDigit(board,i,j):
    if board[i][j] in ['X', '.']:
        return False
    return True

#this function calculates the possible values for a given empty position. 

def getPossibleValues(board,i,j,h_const,v_const):
    
#we need to determine how many free spaces are on the board to know how many need to be filled

    h_free_spaces = 0
    v_free_spaces = 0  
    
#the sums represent the added numbers which can potentially be equal to the constraints

    h_sum = 0
    v_sum = 0
    digits = range(1, 10)
    for o in range(len(board)):
        
#determines when a number should be added to the sum or when the number of free spaces should be increased

        if isNotDigit(board, i, o):
            h_sum += board[i][o]
        elif board[i][o] == '.':
            h_free_spaces += 1   
        if isNotDigit(board, o, j):
            v_sum += board[o][j]
        elif board[o][j] == '.':
            v_free_spaces += 1
            
#this code helps determine which combinations in a given row or column are valid to add to any 
#given vertical or horizontal constraint. 

    target_sum_h = h_const[i - 1][0] - h_sum
    possible_combinations_h = itertools.combinations(digits, h_free_spaces)
    valid_combinations_h = [combo for combo in possible_combinations_h if sum(combo) == target_sum_h]
    values_h = []
    
#this code creates the combinations of possible digits. 

    if valid_combinations_h:
        for n in valid_combinations_h:
            for o in range(len(n)):
                values_h.append(n[o])
    target_sum_v = v_const[j - 1][0] - v_sum
    possible_combinations_v = itertools.combinations(digits, v_free_spaces)
    valid_combinations_v = [combo for combo in possible_combinations_v if sum(combo) == target_sum_v]
    values_v = []
    if valid_combinations_v:
        for nums in valid_combinations_v:
            for o in range(len(nums)):
                values_v.append(nums[o])
                
#this section creates a list with the possible combination sums set to a constraint

    combinedLists = []
    for hValue in values_h:
        if hValue in values_v:
            combinedLists.append(hValue)
    return combinedLists

#this minifunction gets the mrv from the mrv dictionary that contains all
#of the possible combinations for a sum. 

def sortDict(mrvDict):
    return len[cell]

#this function creates the dictionary based on the given constraints and possible cells that have to be filled. 
#it sorts the values based on length and returns the first value, thereby minimiizing backtracking that has to be done. 

def mrv(board,cells, h_const, v_const):
    
# Given a cells list, sort them so that the one with the least possible values, appears first (index 0)
    mrvDict = {}   
    for cell in cells:
        mrvDict[cell] = len(getPossibleValues(board,cell[0],cell[1],h_const,v_const))
    sorted_keys = [key for key, value in sorted(mrvDict.items(), key=lambda item: item[1])]
    return sorted_keys

#this function solves the kakuro board by implementing the functions we've seen above

    
def solveKakuro(board, h_constraints, v_constraints):
    counter = 0
    emptyCells = returnEmptyPosition(board)
    
#this bit returns the board once the problem has been solved because there
#are no empty cells left. 

    if len(emptyCells) == 0:
        return board
    
#this call assigns the dictionary from mrv to the cells which need to be filled.

    emptyCells = mrv(board, emptyCells, h_const, v_const)
    cell = emptyCells.pop(0)
    stack = []
    stack.append((board, cell))
    
#while the stack will the empty cells isn't empty the board will try to get possible values
#generated by the getPossibleValues function. 

    while stack != []:
        board, (i, j) = stack.pop(0)
        possibleValues = getPossibleValues(board, i, j, h_constraints, v_constraints)

#we create a temporary board that saves the state of the puzzle to perform backtracking in case a candidate
#doesn't call the right solution.this function also saves the number of times backtracking occurs and 
#prints it out along with the solution. 

        for digit in possibleValues: 
            if canPlaceDigit(board,i,j,digit, h_constraints, v_constraints):
                tempBoard = deepcopy(board)
                tempBoard[i][j] = digit
                emptyCells = returnEmptyPosition(tempBoard)
                emptyCells = mrv(tempBoard, emptyCells, h_const, v_const)
        
                if emptyCells != []:
                    stack.append((tempBoard, emptyCells.pop(0)))
                else:
                    if isComplete(tempBoard):
                        print("Backtracks: ", counter)
                        return tempBoard
                    else:
                        counter += 1
                        continue                
            else:
                counter += 1
                continue
                
    return None
    

In [7]:
#recursive way to solve kakuro

#unlike the iterative solution, this solution does not implement
#itertools because the backtracking is implicit.
#it will only return the board if it is complete and correctly solved. 

def solveKakuro(board, h_constraints, v_constraints):
    if isComplete(board):
        return board
    else:
        for i in range(len(board)):
            for j in range(len(board[0])):
                if board[i][j] in ['.']:
                    digits = [1,2,3,4,5,6,7,8,9]
                    for digit in digits:
                        if canPlaceDigit(board,i,j,digit, h_constraints, v_constraints):
                            board[i][j] = digit
                            if solveKakuro(board, h_constraints, v_constraints):
                                return board
                    board[i][j] = '.'
                     
    return None

#kakuro can't be solved if there is an empty space waiting for a digit
 
def isComplete(board):
    for row in board:
        if '.' in row:
            return False
    return True
     
#a digit can't be placed in a wall or in a spot that already has a set digit 

def isNotDigit(board,i,j):
    if board[i][j] in ['X', '.']:
        return False
    return True

#just like the iterative version, we initialize the number of free spaces and the 
#potential sum to check whether a constraint's value is met and that row/column is solved. 

def canPlaceDigit(board,i,j,digit,h_const,v_const):
    
#determine if a digit can be placed in a spot on the board

    for o in range(len(board)):
        if board[i][o] == digit or board[o][j] == digit:
            return False
        
#initialize the variables

    h_free_spaces = 0
    h_sum = 0
    v_free_spaces = 0
    v_sum = 0
    
    
#determine if a space is empty or if a digit can be summed   

    for o in range(len(board)):
        if isNotDigit(board, i, o):
            h_sum += board[i][o]
        elif board[i][o] == '.':
            h_free_spaces += 1   
        if isNotDigit(board, o, j):
            v_sum += board[o][j]
        elif board[o][j] == '.':
            v_free_spaces += 1
        
#determine if adding a final number to a potential sum gives you the numerical constraint

    if h_free_spaces == 1:
        if h_sum + digit != h_const[i - 1][0]:
            return False
    if v_free_spaces == 1:
        if v_sum + digit != v_const[j - 1][0]:
            return False
    if h_sum + digit > h_const[i - 1][0]:
        return False 
    if  v_sum + digit > v_const[j - 1][0]:
        return False
    
    return True 

In [186]:
smallBoard = [['X', 'X', 'X'], ['X', '.', '.'], ['X', '.', '.']]
h_small = [[12], [10]]
v_small = [[14], [8]]

a = returnEmptyPosition(smallBoard)
print(mrv(smallBoard, a, h_small, v_small))


[(1, 1), (1, 2), (2, 1), (2, 2)]


In [9]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = solveKakuro(board, h_const, v_const)

# We print the solution in a way to make it easy to debug and check
print('Solution')
pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', '.', '.', '.']],
 'h_constraints': [[13], [10], [21], [18]],
 'id': 1,
 'v_constraints': [[4], [11], [30], [17]]}
Input puzzle
       4 11 30 17
    X  X  X  X  X
13  X .. .. ..  X
10  X .. .. ..  X
21  X  X .. .. ..
18  X  X .. .. ..
Backtracks:  3
Solution
       4 11 30 17
    X  X  X  X  X
13  X  3  2  8  X
10  X  1  3  6  X
21  X  X  5  7  9
18  X  X  1  9  8
Ignore: not checking values yet
